# Librerías y configuraciones


In [4]:
import pandas as pd

In [5]:
pd.set_option('display.max_colwidth', 100)

---

# Carga de datos

In [106]:
df_genres = pd.read_csv('data/df_genres.csv')
df_item_genre = pd.read_csv('data/df_item_genre.csv')
df_items = pd.read_csv('data/df_items.csv')
df_reviews = pd.read_csv('data/df_reviews.csv')
df_users_items = pd.read_csv('data/df_users_items.csv')
df_users = pd.read_csv('data/df_users.csv')
df_user_id_item_id_price = pd.read_csv("data/df_user_id_item_id_price.csv")
df_developer = pd.read_csv('data/df_developer.csv')

In [154]:
def check(input,df,columna):
    return any(part in input.lower() for part in df[columna])

----

1. def userdata( User_id : str ): 
    Debe devolver 
    * cantidad de dinero gastado por el usuario,  
    * el porcentaje de recomendación en base a reviews.recommend
    * cantidad de items.


In [61]:
def userdata(user_id: str):
    # Si el usuario no existe devuelve: {'dinero_gastado': 0.0, 'porcentaje_de_recomendaciones': nan, 'items_recomendados': 0, 'items_comprados': 0}
    
    user_id = user_id.strip()

    # Dinero gastado
    df_filtrado_dinero = df_user_id_item_id_price[df_user_id_item_id_price.user_id == user_id]
    dinero_gastado = round(df_filtrado_dinero.price.sum(),2)

    # Porcentaje de recomendaciones
    df_filtrado_recomendaciones = df_reviews[df_reviews.user_id == user_id]
    
    if df_filtrado_recomendaciones.shape[0] == 0:
         porcentaje_de_recomendacion = 0
    else:
        porcentaje_de_recomendacion = round(df_filtrado_recomendaciones.recommend.mean(),3)*100
    
    # items comprados
    items_comprados = int(df_filtrado_dinero.shape[0])
    
    # items recomendados
    items_recomendados = int(df_filtrado_recomendaciones.shape[0])

    out = {"dinero_gastado":dinero_gastado, "porcentaje_de_recomendaciones": porcentaje_de_recomendacion, 'items_recomendados': items_recomendados,'items_comprados':items_comprados}

    return out

In [62]:
userdata('0212030')

{'dinero_gastado': 104.95,
 'porcentaje_de_recomendaciones': 0,
 'items_recomendados': 0,
 'items_comprados': 5}

In [50]:
df_user_id_item_id_price[df_user_id_item_id_price.user_id == '0212030']

,user_id,item_id,price
2345493,0212030,4560,19.99
2345494,0212030,20540,19.99
2345495,0212030,10500,19.99
2345496,0212030,730,14.99
2345497,0212030,8930,29.99


In [51]:
df_reviews[df_reviews.user_id == '0212030']

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review,posted_date


----

2. def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ): 
    * Cantidad de usuarios que realizaron reviews entre las fechas dadas y, 
    * el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [9]:
####### --------------->>>  Función 2

def countreviews (fecha_inicio:(str), fecha_fin):

    df_filtrado = df_reviews[(df_reviews['posted_date'] >= fecha_inicio) & (df_reviews['posted_date'] <= fecha_fin)]
    cant_usuarios = df_filtrado.shape[0]
    porcentaje_de_recomendacion = round(df_filtrado.recommend.mean(),3)*100
    return {"cantidad":cant_usuarios, "porcentaje":porcentaje_de_recomendacion}
    # return df_filtrado

In [10]:
# pruebas
countreviews('2011-11-05' , '2011-11-20')

{'cantidad': 14, 'porcentaje': 92.9}

In [11]:
# el df df_reviews tiene valores nulos en la columna posted_date. Revisar como influyen en la funcion, si los toma en cuenta, si los ignora, etc.
#  se podria agregar un verificador de fechas validas.

---

3. def genre( género : str ): 
    * Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

In [58]:
####### --------------->>>  Función 3

def genre (genero):
    genero_minusculas = genero.lower().strip()
    if genero_minusculas in list(df_genres.name):
        puesto = int(df_genres[df_genres.name == genero_minusculas].reset_index().at[0,'ranking'])
        salida = {"genero": genero_minusculas, "ranking":puesto}
    else:
        puesto = 'No se encuentra. Intente nuevamente. Ej: Action'
        salida = {"genero": genero_minusculas, "ranking":puesto}
    return salida

In [57]:
# Pruebas
print(genre ('strategy'))
print(genre ('action '))
print(genre ('    ACTION'))
print(genre ('strateggggy'))

{'genero': 'strategy', 'ranking': 6}
{'genero': 'action', 'ranking': 1}
{'genero': 'action', 'ranking': 1}
{'genero': 'strateggggy', 'ranking': 'No se encuentra. Intente nuevamente. Ej: Action'}


----


4.
def userforgenre( género : str ): 
        games.genres
        games.id (item_id)
    Top 5 de usuarios
        head(5)
    con más horas de juego en el género dado, 
        item_df.items.playtime_forever
    con su URL (del user) y 
        items_df.user_url
    user_id.
        items_df.user_id

        *tabla genre
        *tabla genre_item
        *tabla items:        
        *tabla user_item:
        *tabla user:

In [14]:
df_item_genre.head(2)

,item_id,genres
0,761140,action
1,761140,casual


In [15]:
df_users_items.head(2)

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,30,7


In [16]:
df_users.head(2)

,Unnamed: 0,user_id,items_count,user_url
0,0,76561197970982479,277,http://steamcommunity.com/profiles/76561197970982479
1,1,js41637,888,http://steamcommunity.com/id/js41637


In [17]:
# -->>>> funcion 4

def userforgenre( genero : str ): 
    genero_min = genero.lower().strip()

    if genero_min in list(df_item_genre.genres):

        lista_de_items_del_genero = list(df_item_genre[df_item_genre.genres == genero_min].item_id)
        df = df_users_items[df_users_items.item_id.isin(lista_de_items_del_genero)]
        top5 = df.groupby('user_id')['playtime_forever'].sum().reset_index()
        top5 = top5.sort_values('playtime_forever',ascending=False).head() # aca ta el user id y las hs de juego
        top5 = top5.merge(df_users[['user_id','user_url']], on='user_id', how='left')
        users_top5 = list(top5.user_id)
        user_url_top5 = list(top5.user_url)

        return {"users_top5":users_top5, "user_url_top5":user_url_top5}

    else:
        mensaje = 'el género ', genero, ' no éxiste. Intente nuevamente. EJ: "Action"'
        return mensaje

In [18]:
userforgenre('action')

{'users_top5': ['Sp3ctre',
  'shinomegami',
  'REBAS_AS_F-T',
  'Terminally-Chill',
  'DownSyndromeKid'],
 'user_url_top5': ['http://steamcommunity.com/id/Sp3ctre',
  'http://steamcommunity.com/id/shinomegami',
  'http://steamcommunity.com/id/REBAS_AS_F-T',
  'http://steamcommunity.com/id/Terminally-Chill',
  'http://steamcommunity.com/id/DownSyndromeKid']}

-----

5.
def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de salida:
            Ejemplo de salida:
            Activision	
            Año	Contenido Free
            2023	27%
            2022	25%
            xxxx	xx%

            *tabla games
            *tabla games_tags
            *tabla tags

In [68]:
df_developer.head(2)

,developer,price,year
0,Kotoshiro,4.99,2018
1,Secret Level SRL,0.00,2018


In [297]:
def developer(desarrollador : str ):

    desarrollador_minusculas = desarrollador.lower().strip()

    if check(desarrollador_minusculas,df_developer,'developer'):
        
        df_filtrado = df_developer[df_developer.developer == desarrollador_minusculas]
        cantidad_de_items = df_filtrado.shape[0]


        df_xanio = df_filtrado.groupby(['year']).size().reset_index(name=('cantidad_total'))
        df_filtrado_free = df_filtrado[df_filtrado.price == 0].groupby(['year']).size().reset_index(name=('cantidad_free'))
        df = df_xanio.merge(df_filtrado_free,how='left')
        df['porcentaje'] = df.cantidad_free /df.cantidad_total *100
        df['porcentaje'] = df['porcentaje'].fillna(0)


        anios = df_xanio['year'].tolist()
        porcentaje_free_por_anio = df['porcentaje'].tolist()

        out = {
            "cantidad_de_items": cantidad_de_items,
            "anios": anios,
            "porcentaje_free_por_anio": porcentaje_free_por_anio
        }
    else:
         out = (F'El desarrollador {desarrollador}, no éxiste. Intente nuevamente. EJ: "kotoshiro"')
    return out

In [299]:
developer('trickjump games ltd')

{'cantidad_de_items': 1, 'anios': [2018], 'porcentaje_free_por_anio': [0.0]}

In [147]:
df_developer

,developer,price,year
0,kotoshiro,4.99,2018
1,secret level srl,0.00,2018
2,poolians.com,0.00,2017
3,彼岸领域,0.99,2017
4,trickjump games ltd,3.99,2018
...,...,...,...
27588,bidoniera games,1.99,2018
27589,"nikita ""ghost_rus""",1.99,2018
27590,sacada,4.99,2018
27591,laush dmitriy sergeevich,1.99,2018


In [ ]:
# df_items.groupby('developer')['item_id'].count().reset_index()
# item_id_playtime_forever = item_id_playtime_forever.groupby('item_id')['playtime_forever'].sum().reset_index()

In [188]:
df = df_developer[df_developer.developer == 'sacada']
df

,developer,price,year
8522,sacada,4.99,2017
8631,sacada,7.99,2017
8675,sacada,1.99,2017
8987,sacada,7.99,2017
9392,sacada,9.99,2017
9579,sacada,9.99,2017
9977,sacada,3.99,2017
10215,sacada,8.99,2017
12145,sacada,7.99,2017
12479,sacada,9.99,2017


In [203]:
asfaf = df.groupby('year')['price'].count().to_dict()

In [202]:
asfaf


{2017: 17, 2018: 1}

In [213]:
df = df.groupby(['year']).size().reset_index(name='cantidad_items')
df


,year,cantidad_items
0,2017,1
1,2018,1


In [210]:
df_free = df[df.price ==0]

In [211]:
df_free.groupby(['year']).size().reset_index(name='cantidad_items')


,year,cantidad_items
0,2017,1


In [214]:
df/df_free

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [207]:
df.groupby(['developer', 'year']).size() / estadisticas_por_empresa['cantidad_items']) * 100


developer  year
sacada     2017    2
           2018    1
dtype: int64

In [ ]:
    estadisticas_por_empresa = contenido_pago.groupby(['developer', 'year']).size().reset_index(name='cantidad_items')
    estadisticas_por_empresa['porcentaje_free'] = (contenido_gratuito.groupby(['developer', 'year']).size() / estadisticas_por_empresa['cantidad_items']) * 100
    

In [204]:
import pandas as pd

def calcular_estadisticas_por_empresa(df):
    # Convertir 'price' a numérico, considerando errores como NaN
    df['price'] = pd.to_numeric(df['price'], errors='coerce')
    
    # Filtrar solo filas con valores no nulos en 'price' (que indican contenido no gratuito)
    contenido_pago = df.dropna(subset=['price'])
    
    # Filtrar solo filas con valores nulos en 'price' (que indican contenido gratuito)
    contenido_gratuito = df[df['price'].isnull()]
    
    # Calcular cantidad de items y porcentaje de contenido gratuito por año para cada empresa
    estadisticas_por_empresa = contenido_pago.groupby(['developer', 'year']).size().reset_index(name='cantidad_items')
    estadisticas_por_empresa['porcentaje_free'] = (contenido_gratuito.groupby(['developer', 'year']).size() / estadisticas_por_empresa['cantidad_items']) * 100
    
    return estadisticas_por_empresa

# Ejemplo de uso con tu DataFrame
data = {
    'developer': ['sacada', 'sacada', 'sacada'],
    'price': ['0', '9.99', '4.99'],
    'year': ['2017', '2017', '2018']
}

df = pd.DataFrame(data)

estadisticas = calcular_estadisticas_por_empresa(df)
print(estadisticas)


ValueError: cannot join with no overlapping index names